[View in Colaboratory](https://colab.research.google.com/github/ahracho/TIL/blob/master/Fluent_Python/10_Sequence_Hash_Slice.ipynb)

## 10.시퀀스 해킹, 해시, 슬라이스

- 기본 시퀀스 프로토콜 : \_\_len\_\_(), \_\_getitem\_\_() 메서드
- 여러 항목을 가진 객체를 안전하게 표현
- 슬라이싱을 지원해서 새로운 벡터 객체 생성
- 포함된 요소 값을 모두 고려한 집합 해싱
- 커스터마이즈된 포맷 언어 확장


### 10.1. Vector : 사용자 정의 시퀀스형

상속이 아니라 구성을 이용해서 벡터를 구현하는 것이 목표이다. 요소들을 실수형 배열에 저장하고, 벡터가 불변 균일 시퀀스처럼 동작하게 만들 수 있는 메서드를 구현한다.  

### 10.2. Vector 버전 #1 : Vector2d 호환



In [0]:
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1] # Vector(array('d', [3.0, 4.0])) 모양에서 array('d', ) 부분 삭제
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)
   

`reprlib.repr()`는 생략 기호를 이용해서 생성할 문자열의 길이를 제한하여 대형 구조체나 재귀적 구조체도 안전하게 표현한다.   

Vector 클래스가 Vector2d를 상속받도록 구현할 수 있었지만 1) 생성자가 호환되지 않고 2) Vector 클래스가 시퀀스 프로토콜을 구현하는 독자적인 예제를 만들기 위해 상속하지 않았다.  


### 10.3. 프로토콜과 덕 타이핑

파이썬에서는 시퀀스형을 만들기 위해 어떤 특별한 클래스를 상속할 필요 없이 시퀀스 프로토콜에 따르는 메서드를 구현하기만 하면 된다. 예를 들어, 파이썬의 시퀀스 프로토콜은 \_\_len\_\_(), \_\_getitem\_\_() 이다.  

In [0]:
# 이 클래스는 코드 상 시퀀스 프로토콜을 따른다고 정의한 곳은 없지만
# 시퀀스처럼 동작하기 때문에 시퀀스이다. -> 덕타이핑!!

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()
  
  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    
  def __len__(self):
    return len(self._cards)
  
  
  def __getitem__(self, position):
    return self._cards[position]

### 10.4. Vector 버전 #2 : 슬라이스 가능한 시퀀스

FrenchDeck 예제에서 self.\_components를 사용한 것처럼 객체 안에 들어 있는 시퀀스 속성에 위임하면 len, getitem을 다음과 같이 구현할 수 있다.  

In [0]:
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1] # Vector(array('d', [3.0, 4.0])) 모양에서 array('d', ) 부분 삭제
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)
   
  
  def __len__(self):
    return len(self._components)
  
  
  def __getitem__(self, index):
    return self._components[index]

In [8]:
v1 = Vector([3, 4, 5])

print(len(v1))

print(v1[0], v1[-1])

v7 = Vector(range(7))
print(v7[1:4])

3
3.0 5.0
array('d', [1.0, 2.0, 3.0])


Vector의 슬라이스도 배열이 아니라 Vector 객체가 되도록 수정하는 게 좋겠다. Vector를 슬라이싱해서 생성된 배열은 Vector의 기능을 상실하기 때문이다. Vector를 슬라이싱해서 Vector 객체를 생성하려면, 슬라이싱 연산을 배열에 위임하지 않고 \_\_getitem\_\_ 메서드가 받은 인수를 분석해서 제대로 처리하도록 해야 한다.  

#### 10.4.1. 슬라이싱의 작동 방식

In [9]:
class MySeq:
  def __getitem__(self, index):
    return index
  
  
s = MySeq()

print(s[1])
print(s[1:4]) # 슬라이스 객체!
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [11]:
print(slice) # slice는 내장된 자료형이다!

print(dir(slice))

print(help(slice.indices))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.

None


In [12]:
print(slice(None, 10, 2).indices(5)) # 길이가 5인 시퀀스로 처리하겠다
print(slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


#### 10.4.2. 슬라이스를 인식하는 \_\_getitem\_\_()

위의 성질을 이용해서 getitem을 수정

In [0]:
def __getitem__(self, index):
  cls = type(self)
  if isinstance(index, slice):
    return cls(self._components[index])
  elif isinstance(index, numbers.Integral):
    return self._components[index]
  else:
    msg = '{cls.__name__} indices must be integrals'
    raise TypeError(msg.format(cls=cls))

### 10.5.  Vector 버전 #3 : 동적 속성 접근

v.x, v.y 처럼 벡터 요소를 이름으로 접근할 수 있도록 수정해보자. @property 데커레이터를 이용해서 ,x, y에 읽기 전용 접근을 제공할 수 있지만 여기서는 \_\_getattr\_\_() 특별 메서드를 사용한다.

속성을 찾지 못하면 인터프리터는 \_\_getattr\_\_() 메서드를 호출한다. 예를 들어, my_obj.x가 주어지면 먼저 my_obj에 x 속성이 있는지 검사한다. 속성이 없으면 이 객체의 클래스에서 더 검색한다. 그래도 없으면 상속 그래프를 거슬러 올라가면서 x 속성을 찾고 그래도 찾지 못하면 self와 속성명을 문자열로 전달해서 my_obj 클래스에 정의된 \_\_getattr\_\_()를 호출한다.  

In [0]:
shortcut_names = 'xyzt'

def __getattr__(self, name):
  cls = type(cls)
  if len(name) == 1:
    pos = cls.shortcut_names.find(name)
    if 0 <= pos < len(self._components):
      return self._components[pos]
    
  msg = '{.__name__!r} object has no attribute {!r}'
  raise AttributeError(msg.format(cls, name))

이렇게 구현할 경우, getitem은 성공적으로 이루어지지만, 새로운 값을 할당하였을 경우 값이 변경되지 않는 문제가 있다. 

> v.x -> 0.0  
> v.x = 10  
> v.x -> 10.0  
> v -> Vector([0.0, 1.0, 2.0])  

속성이 없을 때만 \_\_getattr\_\_()를 부르기 때문에 v.x에 값을 할당하게 되면 새 속성이 생기면서 이후에는 \_\_getattr\_\_()이 안 불리는 것. \_\_setattr\_\_도 같이 수정해주어야.

In [0]:
def __setattr__(self, name, value):
  cls = type(self)
  if len(name) == 1:
    if name in cls.shortcut_names:
      error = 'readonly attribute {attr_name!r}'
    elif name.islower():
      error = "can't set attributes 'a' to 'z' in {cls_name!r}"
    else:
      error =''
    if error:
      msg = error.format(cls_name=cls.__name__, attr_name=name)
      raise AttributeError(msg)
  super().__setattr__(name, value)

### 10.6. Vector 버전 #4 : 해싱 및 더 빠른 ==

\_\_hash\_\_() 메서드를 구현하면 Vector 객체를 해시할 수 있게 된다. Hash 값을 계산할 때는 reduce 함수를 사용하면 편하다.  

In [0]:
import operator
functools.reduce(operator.xor, range(6))

In [0]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
  typecode = 'd'
  # 중간 코드 생략
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __hash__(self):
    hashes = (hash(x) for x in self._components)
    return functools.reduce(operator.xor, hashes, 0)
  
  # 맵-리듀스 계산
  # 맵 단계에서는 각 요소에 대한 해시를 계산하고, 리듀스 단계에서는 xor 연산을 적용한다. 
  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)

Vector 요소가 적으면 상관없지만 요소가 많아지면 other를 복사하여 비교하기 때문에 비효율적이다. 

In [0]:
def __eq__(self, other):
  if len(self) != len(other):
    return False
  
  for a, b in zip(self, other):
    if a != b:
      return False
    
  return True


def __eq__(self, other):
  return len(self) == len(other) and all(a==b for a, b in zip(self, other))

### 10.7. Vector 버전 #5 : 포매팅

In [0]:
def angle(self, n):
  r = math.sqrt(sum(x*x for x in self[n:]))
  a = math.atan2(r, self[n-1])
  if (n == len(self) -1) and (self[-1] < 0):
    return math.pi*2 -a
  else:
    return a
  
def angles(self):
  return (self.angle(n) for n in ragne(1, len(self)))

def __format__(self, fmt_spec=''):
  if fmt_spec.endswith('h'):
    fmt_spec = fmt_spec[:-1]
    coords = itertools.chain([abs(self)], self.angles())
    outer_fmt = '<{}>'
  else:
    coords = self
    outer_fmt = '({})'
    
  components = (format(c, fmt_spec) for c in coords)
  return outer_fmt.format(', '.join(components))